In [104]:
import numpy as np

In [105]:
# 收集数据：可以使用任何方法
def loadDataSet():
    """
    创建数据集
    :return: 单词列表postingList，所属类别classVec
    """
    postingList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                   ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0, 1, 0, 1, 0, 1]
    return postingList, classVec

In [106]:
# 准备数据：从文本中构建词向量
def createVocabList(dataSet):
    """
    获取所有单词的集合
    :param dataSet: 数据集
    :return: 所有单词的集合(即不含重复元素的单词列表)
    """
    vocabSet = set() # create empty set
    for document in dataSet:
        # 操作符 | 用于求两个集合的并集
        vocabSet = vocabSet | set(document) # union of the two sets
    return list(vocabSet)

def setOfWords2Vec(vocabList, inputSet):
    """
    遍历查看该单词是否出现，出现该单词则将该单词置1
    :param vocabList: 所有单词集合列表
    :param inputSet: 输入数据集
    :return: 匹配列表[0, 1, 0, 1...]，其中1与0表示词汇表中的单词是否出现在输入的数据集中
    """
    # 创建一个和词汇表等长的向量，并将其元素都设置为0
    returnVec = [0] * len(vocabList) # [0, 0...]
    # 遍历文档中的所有单词，如果出现了词汇表中的单词，则将输出的文档想两种的对应值设为1
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print("the word: %s is not in my Vocabulary!" % word)
    return returnVec

In [107]:
# 分析数据：检查词条确保解析的正确性
# 检查函数执行情况，检查词表，不出现重复单词，需要的话，可以对其进行排序。
listOPosts, listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)
myVocabList

# 检查函数有效性。例如：myVocabList中索引为2的元素是什么单词？应该是'I'。
# 该单词在第3篇文档中出现了，现在检查一下看看它是否出现在第四篇文档中。
setOfWords2Vec(myVocabList, listOPosts[0])
setOfWords2Vec(myVocabList, listOPosts[3])

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0]

In [108]:
# 训练算法：从词向量计算概率
# 朴素贝叶斯分类器训练函数
def trainNB0(trainMatrix, trainCategory):
    """
    训练数据原版
    :param trainMatrix: 文件单词矩阵 [[1, 0, 1, 1, 1...], [], []...]
    :param trainCayegory: 文件对应的类别 [0, 1, 1, 0...]，列表长度等于单词矩阵书
    :return:
    """
    # 文件数
    numTrainDocs = len(trainMatrix)
    # 单词数
    numWords = len(trainMatrix[0])
    # 侮辱性文件的出现概率，即trainCategory中所有的1的个数，
    # 代表的就是多少个侮辱性文件，与文件的总数相除就得到了侮辱性文件的出现概率
    pAbusive = sum(trainCategory) / float(numTrainDocs)
    # 构造单词出现次数列表
    p0Num = np.zeros(numWords)
    p1Num = np.zeros(numWords)
    
    # 整个数据集单词出现总数
    p0Denom = 0.0
    p1Denom = 0.0
    for i in range(numTrainDocs):
        # 是否是侮辱性文件
        if trainCategory[i] == 1:
            # 如果是侮辱性文件，对侮辱性文件的向量进行加和
            p1Num += trainMatrix[i]
            # 对向量中的所有元素进行求和，也就是计算所有侮辱性文件中出现的单词总数
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    # 类别1，即侮辱性文档的 [P(F1|C1), P(F2|C1), ...] 列表
    # 即在类别1下，每个单词出现的概率
    p1Vect = p1Num / p1Denom
    # 类别0下，每个单词出现的概率
    p0Vect = p0Num / p0Denom
    return p0Vect, p1Vect, pAbusive

In [109]:
# 测试算法：根据现实情况修改分类器
def trainNB1(trainMatrix, trainCategory):
    """
    训练数据优化版本
    :param trainMatrix: 文件单词矩阵
    :param trainCayegory: 文件对应的类别
    :return:
    """
    # 总文件数
    numTrainDocs = len(trainMatrix)
    # 总单词数
    numWords = len(trainMatrix[0])
    # 侮辱性文件的出现概率
    pAbusive = sum(trainCategory) / float(numTrainDocs)
    # 构造单词出现次数列表
    # p0Num 正常的统计
    # p1Num 侮辱的统计
    p0Num = np.ones(numWords) # [0, 0, ...] --> [1, 1, ...]
    p1Num = np.ones(numWords)
    
    # 整个数据集单词出现总数，2.0
    # 根据样本/实际调查结果调整分母的值（2主要是避免分母为0，当然值可以调整）
    # p0Denom 正常的统计
    # p1Denom 侮辱的统计
    p0Denom = 2.0
    p1Denom = 2.0
    for i in range(numTrainDocs):
        # 是否是侮辱性文件
        if trainCategory[i] == 1:
            # 如果是侮辱性文件，对侮辱性文件的向量进行加和
            p1Num += trainMatrix[i]
            # 对向量中的所有元素进行求和，也就是计算所有侮辱性文件中出现的单词总数
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    # 类别1，即侮辱性文档的 [log(P(F1|C1)), log(P(F2|C1)), ...] 列表
    # 即在类别1下，每个单词出现的概率
    p1Vect = np.log(p1Num / p1Denom)
    # 类别0下，每个单词出现的概率
    p0Vect = np.log(p0Num / p0Denom)
    return p0Vect, p1Vect, pAbusive

In [110]:
# 使用算法：对社区留言板言论进行分类
# 朴素贝叶斯分类函数
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    """
    使用算法：
        # 将乘法转换为加法
        乘法：P(C|F1F2...Fn) = p(F1F2...Fn|C)P(C)/P(F1F2...Fn)
        加法：P(F1|C)*P(F2|C)...P(Fn|C)P(C) --> log(P(F1|C))+log(P(F2|C))+...+log(P(Fn|C))+log(P(C))
    :param vec2Classify: 待测数据[0, 1, 1, 1, 1, ...]，即要分类的向量
    :param p0Vec: 类别0，即正常文档的[log(P(F1|C0)), log(P(F2|C0)), ...]列表
    :param p1Vec: 类别1，即侮辱性文档的[log(P(F1|C1)), log(P(F2|C1)), ...]列表
    :param pClass1: 类别1，即侮辱性文件的出现概率
    :return: 类别1或0
    """
    # 计算公式 log(P(F1|C))+log(P(F2|C))+...+log(P(Fn|C))+log(P(C))
    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1) # 即贝叶斯准则的分子
    p0 = sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0

def testingNB():
    """
    测试朴素贝叶斯算法
    """
    # 1. 加载数据集
    listOPosts, listClasses = loadDataSet()
    # 2. 创建单词集合
    myVocabList = createVocabList(listOPosts)
    # 3. 计算单词是否出现并创建数据矩阵
    trainMat = []
    for postinDoc in listOPosts:
        # 返回 m*len(myVocabList) 的矩阵，记录的都是0，1信息
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    # 4. 训练数据
    p0V, p1V, pAb = trainNB1(np.array(trainMat), np.array(listClasses))
    # 5. 测试数据
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, 'classified as: ', classifyNB(thisDoc, p0V, p1V, pAb))
    testEntry = ['stupid', 'garbage']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, 'classified as: ', classifyNB(thisDoc, p0V, p1V, pAb))

In [111]:
testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1
